# 5. Разностный метод для ОДУ первого порядка. Метод разностной прогонки.

$$ -p(x)u`` + q(x)u` + r(x)u = f(x) $$
$$p(x) = \frac{7+x}{8-3x}$$
$$q(x) = 1 - \frac{x}{3}$$
$$r(x) = \frac{1}{3}ln(3+x)$$
$$f(x) = \frac{1+x}{2}$$
Граничные условия:
$$\alpha_1 u(a) - \alpha_2 u`(a) = \alpha$$
$$\beta_1 u(b) + \beta_2 u`(b) = \beta$$
$$a = -1, \alpha_1 = 3, \alpha_2 = 1, \alpha = 0$$
$$b = 1, \beta_1 = 0, \beta_2 = 1, \beta = 0$$

In [137]:
from math import log
from pandas import DataFrame as df
from scipy import interpolate

In [138]:
def p(x):
    return (7+x)/(8-3*x)

def q(x):
    return 1 - x/3

def r(x):
    return log(3+x)/3

def f(x):
    return (1+x)/2

a = -1
alpha1 = 3
alpha2 = 1
alpha = 0
b = 1
beta1 = 0
beta2 = 1
beta = 0

In [139]:
def getA(x, h):
    return -p(x)/(h**2) - q(x)/(2*h)

def getB(x, h):
    return -2*p(x)/(h**2) - r(x)

def getC(x, h):
    return -p(x)/(h**2) + q(x)/(2*h)

def getG(x):
    return f(x)    

In [140]:
# Точность O(h^(precision)), где precision \in {1, 2}
def solve(n, precision):
    h = 2 / n
    
    if precision == 1:
        x = [a + i * h for i in range(n + 1)]
    else:
        x = [a - h/2 + i * h for i in range(n + 2)]
    
    A = [0] * len(x)
    B = [0] * len(x)
    C = [0] * len(x)
    G = [0] * len(x)
    
    if precision == 1:
        B[0] = -alpha1 + alpha2 / h
        C[0] = -alpha2 / h
        G[0] = alpha
        A[n] = -beta2 / h
        B[n] = -beta1 - beta2 / h
        G[n] = beta
    else:
        B[0] = -alpha1 / 2 - alpha2 / h
        C[0] = alpha1 / 2 - alpha2 / h
        G[0] = alpha
        A[n+1] = beta1 / 2 - beta2 / h
        B[n+1] = -beta1 / 2 - beta2 / h
        G[n+1] = beta
                
    for i in range(1, len(x) - 1):
        A[i] = getA(x[i], h)
        B[i] = getB(x[i], h)
        C[i] = getC(x[i], h)
        G[i] = getG(x[i])
        
    s = [0] * len(x)
    t = [0] * len(x)
    y = [0] * len(x);
    
    s[0] = C[0] / B[0]
    for i in range(1, len(x)):
        s[i] = C[i] / (B[i] - A[i] * s[i-1])
    
    t[0] = -G[0] / B[0]
    for i in range(1, len(x)):
        t[i] = (A[i] * t[i-1] - G[i]) / (B[i] - A[i] * s[i-1])
        
    y[len(x) - 1] = t[len(x) - 1]
    for i in range(len(x) - 2, -1, -1):
        y[i] = s[i] * y[i+1] + t[i]
        
    if n == 10:
        table = df(data={
            'x': x,
            'A': A,
            'B': B,
            'C': C,
            'G': G,
            's': s,
            't': t,
            'y': y
        })
        table.style.hide_index()
        print(table)
    return x, y

In [141]:
_, y10 = solve(10, 1)
_, y20 = solve(20, 1)

# Уточнение по Ричардсону
y_ut_1 = [0] * len(y10)
for i in range(len(y_ut_1)):
    y_ut_1[i] = y20[2 * i] + (y20[2 * i] - y10[i])

      x          A          B          C    G         s         t         y
0  -1.0   0.000000   2.000000  -5.000000  0.0 -2.500000  0.000000 -0.027076
1  -0.8 -18.070513 -30.070511 -11.737179  0.1  0.155982  0.001329  0.010831
2  -0.6 -19.326531 -32.944884 -13.326531  0.2  0.445252  0.007540  0.060915
3  -0.4 -20.768116 -36.188069 -15.101449  0.3  0.560537  0.016948  0.119875
4  -0.2 -22.434109 -39.878090 -17.100775  0.4  0.626335  0.028576  0.183621
5   0.0 -24.375000 -44.116204 -19.375000  0.5  0.671594  0.041476  0.247543
6   0.2 -26.657658 -49.036366 -21.990991  0.6  0.706351  0.054785  0.306833
7   0.4 -29.372549 -54.819690 -25.039216  0.7  0.734883  0.067773  0.356831
8   0.6 -32.645161 -61.717301 -28.645161  0.8  0.759276  0.079849  0.393339
9   0.8 -36.654762 -70.087857 -32.988095  0.9  0.780658  0.090562  0.412880
10  1.0  -5.000000  -5.000000   0.000000  0.0 -0.000000  0.412880  0.412880


In [142]:
# получает значения в точках основной сетки
def align(n, x, y):
    h = 2 / n
    p = interpolate.interp1d(x, y)
    X = [a + i * h for i in range(n+1)]
    Y = p(X)
    return X, Y

x, y10 = align(10, *solve(10, 2))
_, y20 = align(20, *solve(20, 2))

y_ut_2 = [0] * len(y10)
for i in range(len(y_ut_2)):
    y_ut_2[i] = y20[2 * i] + (y20[2 * i] - y10[i])

      x          A          B          C     G         s         t         y
0  -1.1   0.000000  -6.500000  -3.500000  0.00  0.538462 -0.000000  0.033225
1  -0.9 -17.502336 -28.751985 -11.002336  0.05  0.569254  0.002587  0.061703
2  -0.7 -18.677393 -31.465755 -12.510726  0.15  0.600508  0.009519  0.103849
3  -0.5 -20.021930 -34.515957 -14.188596  0.25  0.630811  0.019588  0.157083
4  -0.3 -21.570225 -37.971533 -16.070225  0.35  0.659567  0.031707  0.217965
5  -0.1 -23.366466 -41.921169 -18.199799  0.45  0.686541  0.044923  0.282395
6   0.1 -25.468615 -46.481030 -20.635281  0.55  0.711665  0.058426  0.345898
7   0.3 -27.954225 -51.806425 -23.454225  0.65  0.734957  0.071548  0.403942
8   0.5 -30.929487 -58.109895 -26.762821  0.75  0.756481  0.083751  0.452263
9   0.7 -34.543785 -65.690348 -30.710452  0.85  0.776327  0.094621  0.487141
10  0.9 -39.014151 -74.981961 -35.514151  0.95  0.794603  0.103851  0.505612
11  1.1  -5.000000  -5.000000   0.000000  0.00 -0.000000  0.505612  0.505612

In [143]:
# wolfram: https://www.wolframalpha.com/input/?i=solve+%7B-%287%2Bx%29%2F%288-3x%29y%60%60%28x%29+%2B+%281+-+x%2F3%29y%60%28x%29+%2B+ln%283%2Bx%29y%28x%29%2F3+%3D+%281%2Bx%29%2F2%2C+y%60%28-1%29-3y%28-1%29%3D0%2C+y%60%281%29%3D0%7D+Runge-Kutta+method+x%3D-1..1
# решение в вольфраме получено методом Рунге-Кутты 4 порядка
y_wolfram = [0.047211, 0.0824656, 0.130122, 0.187183, 0.249883, 0.313931, 0.37481, 0.428106, 0.469807, 0.496553, 0.505771]

table_data = {
    'x': x,
    '$y_{wolfram}$': y_wolfram,
    '$y_{ut}(h)$': y_ut_1,
    '$|y_{ut} - y_{wolfram}|(h)$': [abs(y_ut_1[i] - y_wolfram[i]) for i in range(len(y_ut_1))],
    '$y_{ut}(h^2)$': y_ut_2,
    '$|y_{ut} - y_{wolfram}|(h^2)$': [abs(y_ut_2[i] - y_wolfram[i]) for i in range(len(y_ut_2))]
}

table = df(data=table_data)
table.style.hide_index()
table.style.set_properties(subset=list(table_data.keys()), width='140px')

,x,$y_{wolfram}$,$y_{ut}(h)$,$|y_{ut} - y_{wolfram}|(h)$,$y_{ut}(h^2)$,$|y_{ut} - y_{wolfram}|(h^2)$
0,-1.000000,0.047211,0.007145,0.040066,0.046721,0.000490
1,-0.800000,0.082466,0.043816,0.038650,0.081811,0.000655
2,-0.600000,0.130122,0.092951,0.037171,0.129347,0.000775
3,-0.400000,0.187183,0.151519,0.035664,0.186351,0.000832
4,-0.200000,0.249883,0.215715,0.034168,0.249064,0.000819
5,0.000000,0.313931,0.281202,0.032729,0.313189,0.000742
6,0.200000,0.374810,0.343418,0.031392,0.374198,0.000612
7,0.400000,0.428106,0.397902,0.030204,0.427656,0.000450
8,0.600000,0.469807,0.440602,0.029205,0.469532,0.000275
9,0.800000,0.496553,0.468127,0.028426,0.496447,0.000106
